# Displaying all individual planes of a testbed (here: THD2)

To make this notebook run, set the variable `save_dir` to wherever you want the output data saved, and create the sub-directories `direct_propagation` and `coro_propagation` inside it.

Set `propagate = True` and run the entire testbed propagation. To prevent cluttering of your directory, you can then set `propagate = False` - but it won't overwrite the data in any case.

As you move on through the notebook, you will have to change the individual fits file names that are read in for plotting.

In [ ]:
save_dir = '/Users/ilaginja/Documents/LESIA/THD/asterix_model_matching/wrapped-vortex_700nm_samp646'
propagate = True

In [ ]:
# Imports
%matplotlib inline
import os
from astropy.io import fits
import matplotlib as mpl
from matplotlib.colors import LogNorm, TwoSlopeNorm
import matplotlib.pyplot as plt
import numpy as np

from Asterix import Asterix_root
from Asterix.utils import get_data_dir, read_parameter_file
from Asterix.optics import Pupil, Coronagraph, DeformableMirror, Testbed
from Asterix.main_THD import THD2

In [ ]:
# Some setup for pretty plotting
mpl.rc('image', origin='lower',   # Put the origin in the lower left corner.
       interpolation=None)        # Do not interpolate between pixels in the display.

cenm = TwoSlopeNorm(vcenter=0)    # Create normalization where 0 is always the center of your cmap

In [ ]:
# Load the template parameter file
parameter_file_ex = os.path.join(Asterix_root, "Example_param_file.ini")
config = read_parameter_file(parameter_file_ex)

In [ ]:
thd2 = THD2(parameter_file_ex)

### Add amplitude aberrations (i.e., actuator print through)

In [ ]:
data_dir = get_data_dir()
model_local_dir = os.path.join(data_dir, 'Model_local')

simu_config = config["SIMUconfig"]
ampl_abb_up = thd2.generate_ampl_aberr(simu_config, Model_local_dir=model_local_dir)
input_wavefront = thd2.EF_from_phase_and_ampl(ampl_abb=ampl_abb_up)

## Propagation through testbed

Run a propagation with the FPM in and the FPM out and spit out all files to disk before reading them back in and plotting them.

Note how some planes only spit out the real part, while others spit out separate files for real and imaginary part.

In [ ]:
print("############################")
print(f"# Coronagraph used: {thd2.corono.corona_type} #")
print("############################")
print(f"Wavelength: {thd2.wavelength_0} m = {thd2.wavelength_0 * 1e9} nm")

In [ ]:
dpath = os.path.join(save_dir, 'direct_propagation')
cpath = os.path.join(save_dir, 'coro_propagation')

In [ ]:
if propagate:
    direct = thd2.todetector_intensity(entrance_EF=input_wavefront, in_contrast=True, noFPM=True,
                                       dir_save_all_planes=dpath)
    coro = thd2.todetector_intensity(entrance_EF=input_wavefront, in_contrast=True, noFPM=False,
                                     dir_save_all_planes=cpath)

## Direct propagation

All planes when no FPM is in.

### Pupil mask

In [ ]:
pup_after_real = fits.getdata(os.path.join(dpath, f'13_50_47_784_EF_PP_after_pupil_wl783_RE.fits'))
print(pup_after_real.shape)

plt.imshow(pup_after_real, cmap='Greys_r')
plt.title('Entrance pupil')
plt.colorbar()

### DM plane

In [ ]:
# DM1
dm1_before_real = fits.getdata(os.path.join(dpath, '13_50_47_833_EF_PP_before_DM1_wl783_RE.fits'))
dm1_phase = fits.getdata(os.path.join(dpath, '13_50_47_841_phase_DM1_wl783_RE.fits'))

print(dm1_before_real.shape)
print(dm1_phase.shape)

plt.figure(figsize=(14, 7))
plt.subplot(1, 2, 1)
plt.imshow(dm1_before_real, cmap='Greys_r')
plt.title('Before DM1')
plt.colorbar()
plt.subplot(1, 2, 2)
plt.imshow(dm1_phase, cmap='RdBu', norm=cenm)
plt.title('DM1 phase')
plt.colorbar()

In [ ]:
# DM3
dm3_before_real = fits.getdata(os.path.join(dpath, '13_50_47_849_EF_PP_before_DM3_wl783_RE.fits'))
dm3_phase = fits.getdata(os.path.join(dpath, '13_50_47_854_phase_DM3_wl783_RE.fits'))

print(dm3_before_real.shape)
print(dm3_phase.shape)

plt.figure(figsize=(14, 7))
plt.subplot(1, 2, 1)
plt.imshow(dm3_before_real, cmap='Greys_r')
plt.title('Before DM3')
plt.colorbar()
plt.subplot(1, 2, 2)
plt.imshow(dm3_phase, cmap='RdBu', norm=cenm)
plt.title('DM3 phase')
plt.colorbar()

### FPM plane

In [ ]:
# Before the FPM
fpm_before = fits.getdata(os.path.join(dpath, '13_50_47_876_EF_FP_before_FPM_wl783_RE_and_IM.fits'))
print(fpm_before.shape)
print(type(fpm_before))

plt.figure(figsize=(14, 7))
plt.subplot(1, 2, 1)
plt.imshow(fpm_before[0], cmap='inferno')
plt.title('Real before FPM')
plt.colorbar()
plt.subplot(1, 2, 2)
plt.imshow(fpm_before[1], cmap='RdBu')
plt.title('Imag before FPM')
plt.colorbar()

In [ ]:
# After the FPM
fpm_after = fits.getdata(os.path.join(dpath, '13_50_47_886_EF_FP_after_FPM_wl783_RE_and_IM.fits'))
print(fpm_after.shape)
print(type(fpm_after))

plt.figure(figsize=(14, 7))
plt.subplot(1, 2, 1)
plt.imshow(fpm_after[0], cmap='inferno')
plt.title('Real after FPM')
plt.colorbar()
plt.subplot(1, 2, 2)
plt.imshow(fpm_after[1], cmap='RdBu')
plt.title('Imag after FPM')
plt.colorbar()

### Lyot plane

In [ ]:
# Before the LS
ls_before = fits.getdata(os.path.join(dpath, '13_50_47_901_EF_PP_before_LS_wl783_RE_and_IM.fits'))
print(ls_before.shape)
print(type(ls_before))

plt.figure(figsize=(14, 7))
plt.subplot(1, 2, 1)
plt.imshow(ls_before[0], cmap='inferno', norm=LogNorm())
plt.title('Real before LS')
plt.colorbar()
plt.subplot(1, 2, 2)
plt.imshow(ls_before[1], cmap='RdBu', norm=cenm)
plt.title('Imag before LS')
plt.colorbar()

In [ ]:
# The LS itself
ls_real = fits.getdata(os.path.join(dpath, '13_50_47_904_LS_wl783_RE.fits'))
print(ls_real.shape)
print(type(ls_real))

plt.imshow(ls_real, cmap='Greys_r')
plt.title('Lyot stop')
plt.colorbar()

In [ ]:
# After the LS
ls_after = fits.getdata(os.path.join(dpath, '15_23_38_184_EF_PP_after_LS_wl700_RE_and_IM.fits'))
print(ls_after.shape)
print(type(ls_after))

plt.figure(figsize=(10, 10))
plt.subplot(2, 2, 1)
plt.imshow(ls_after[0], cmap='inferno', norm=LogNorm())
plt.title('Real after LS')
plt.colorbar()
plt.subplot(2, 2, 2)
plt.imshow(ls_after[1], cmap='RdBu', norm=cenm)
plt.title('Imag after LS')
plt.colorbar()

ls_after_complex = ls_after[0] + 1.j * ls_after[1]
print(ls_after_complex.shape)
print(type(ls_after_complex))

plt.subplot(2, 2, 3)
plt.imshow(np.abs(ls_after_complex)**2, cmap='Greys_r', norm=LogNorm())
plt.title('After LS intensity')
plt.colorbar()
plt.subplot(2, 2, 4)
plt.imshow(np.angle(ls_after_complex), cmap='RdBu', norm=cenm)
plt.title('After LS phase')
plt.colorbar()

### Final image (direct)

In [ ]:
img = fits.getdata(os.path.join(dpath, '13_50_47_924_EF_FP_after_THD2_obj_wl783_RE_and_IM.fits'))
img_intensity = fits.getdata(os.path.join(dpath, '13_50_47_932_Int_FP_after_THD2_obj_RE.fits'))
print(img.shape)
print(type(img))

plt.figure(figsize=(10, 10))
plt.subplot(2, 2, 1)
plt.imshow(img[0], cmap='inferno')
plt.title('Real final image')
plt.colorbar()
plt.subplot(2, 2, 2)
plt.imshow(img[1], cmap='RdBu')
plt.title('Imag final image')
plt.colorbar()
plt.subplot(2, 2, 3)
plt.imshow(np.abs(img[0] + 1.j * img[1])**2, cmap='inferno', norm=LogNorm())
plt.title('Intensity from E-field, final image')
plt.colorbar()
plt.subplot(2, 2, 4)
plt.imshow(img_intensity, cmap='inferno', norm=LogNorm())
plt.title('Intensity form file, final image')
plt.colorbar()

## Coronagraph propagation

All planes when the FPM is in the optical system.

### Entrance pupil

In [ ]:
pup_after_real = fits.getdata(os.path.join(cpath, f'13_50_47_936_EF_PP_after_pupil_wl783_RE.fits'))
print(pup_after_real.shape)

plt.imshow(pup_after_real, cmap='Greys_r')
plt.title('Entrance pupil')
plt.colorbar()

### DM plane

In [ ]:
# DM1
dm1_before_real = fits.getdata(os.path.join(cpath, '13_50_47_938_EF_PP_before_DM1_wl783_RE.fits'))
dm1_phase = fits.getdata(os.path.join(cpath, '13_50_47_940_phase_DM1_wl783_RE.fits'))

print(dm1_before_real.shape)
print(dm1_phase.shape)

plt.figure(figsize=(14, 7))
plt.subplot(1, 2, 1)
plt.imshow(dm1_before_real, cmap='Greys_r')
plt.title('Before DM1')
plt.colorbar()
plt.subplot(1, 2, 2)
plt.imshow(dm1_phase, cmap='RdBu')
plt.title('DM1 phase')
plt.colorbar()

In [ ]:
# DM3
dm3_before_real = fits.getdata(os.path.join(cpath, '13_50_47_942_EF_PP_before_DM3_wl783_RE.fits'))
dm3_phase = fits.getdata(os.path.join(cpath, '13_50_47_945_phase_DM3_wl783_RE.fits'))

print(dm3_before_real.shape)
print(dm3_phase.shape)

plt.figure(figsize=(14, 7))
plt.subplot(1, 2, 1)
plt.imshow(dm3_before_real, cmap='Greys_r')
plt.title('Before DM3')
plt.colorbar()
plt.subplot(1, 2, 2)
plt.imshow(dm3_phase, cmap='RdBu')
plt.title('DM3 phase')
plt.colorbar()

### FPM plane

In [ ]:
# Before the FPM
fpm_before = fits.getdata(os.path.join(cpath, '13_50_47_963_EF_FP_before_FPM_wl783_RE_and_IM.fits'))
print(fpm_before.shape)
print(type(fpm_before))

plt.figure(figsize=(14, 7))
plt.subplot(1, 2, 1)
plt.imshow(fpm_before[0], cmap='inferno')
plt.title('Real before FPM')
plt.colorbar()
plt.subplot(1, 2, 2)
plt.imshow(fpm_before[1], cmap='RdBu')
plt.title('Imag before FPM')
plt.colorbar()

In [ ]:
# The FPM itself
# Before the FPM
fpm = fits.getdata(os.path.join(cpath, '13_50_47_971_FPM_wl783_RE_and_IM.fits'))
fpm_complex = fpm[0] + 1.j * fpm[1]
print(fpm_complex.shape)
print(type(fpm_complex))

plt.figure(figsize=(14, 7))
plt.subplot(1, 2, 1)
plt.imshow(np.abs(fpm_complex)**2, cmap='Greys_r')
plt.title('FPM intensity')
plt.colorbar()
plt.subplot(1, 2, 2)
plt.imshow(np.angle(fpm_complex), cmap='RdBu', norm=cenm)
plt.title('FPM phase')
plt.colorbar()

In [ ]:
# After the FPM
fpm_after = fits.getdata(os.path.join(cpath, '13_50_47_993_EF_FP_after_FPM_wl783_RE_and_IM.fits'))
print(fpm_after.shape)
print(type(fpm_after))

plt.figure(figsize=(14, 7))
plt.subplot(1, 2, 1)
plt.imshow(fpm_after[0], cmap='inferno')
plt.title('Real after FPM')
plt.colorbar()
plt.subplot(1, 2, 2)
plt.imshow(fpm_after[1], cmap='RdBu')
plt.title('Imag after FPM')
plt.colorbar()

### Lyot plane

In [ ]:
# Before the LS
ls_before = fits.getdata(os.path.join(cpath, '13_50_48_013_EF_PP_before_LS_wl783_RE_and_IM.fits'))
print(ls_before.shape)
print(type(ls_before))

plt.figure(figsize=(14, 7))
plt.subplot(1, 2, 1)
plt.imshow(ls_before[0], cmap='inferno', norm=LogNorm())
plt.title('Real before LS')
plt.colorbar()
plt.subplot(1, 2, 2)
plt.imshow(ls_before[1], cmap='RdBu', norm=cenm)
plt.title('Imag before LS')
plt.colorbar()

In [ ]:
# The LS itself
ls_real = fits.getdata(os.path.join(cpath, '13_50_48_016_LS_wl783_RE.fits'))
print(ls_real.shape)
print(type(ls_real))

plt.imshow(ls_real, cmap='Greys_r')
plt.title('Lyot stop')
plt.colorbar()

In [ ]:
# After the LS
ls_after = fits.getdata(os.path.join(cpath, '15_23_38_788_EF_PP_after_LS_wl700_RE_and_IM.fits'))
print(ls_after.shape)
print(type(ls_after))

plt.figure(figsize=(10, 10))
plt.subplot(2, 2, 1)
plt.imshow(ls_after[0], cmap='inferno')#, norm=LogNorm())
plt.title('Real after LS')
plt.colorbar()
plt.subplot(2, 2, 2)
plt.imshow(ls_after[1], cmap='RdBu')#, norm=cenm)
plt.title('Imag after LS')
plt.colorbar()

ls_after_complex = ls_after[0] + 1.j * ls_after[1]
print(ls_after_complex.shape)
print(type(ls_after_complex))

plt.subplot(2, 2, 3)
plt.imshow(np.abs(ls_after_complex)**2, cmap='Greys_r', norm=LogNorm())
plt.title('After LS intensity')
plt.colorbar()
plt.subplot(2, 2, 4)
plt.imshow(np.angle(ls_after_complex), cmap='RdBu', norm=cenm)
plt.title('After LS phase')
plt.colorbar()

### Final image (coronagraphic)

In [ ]:
img = fits.getdata(os.path.join(cpath, '13_50_48_030_EF_FP_after_THD2_obj_wl783_RE_and_IM.fits'))
img_intensity = fits.getdata(os.path.join(cpath, '13_50_48_041_Int_FP_after_THD2_obj_RE.fits'))
print(img.shape)
print(type(img))

plt.figure(figsize=(10, 10))
plt.subplot(2, 2, 1)
plt.imshow(img[0], cmap='inferno')
plt.title('Real final image')
plt.colorbar()
plt.subplot(2, 2, 2)
plt.imshow(img[1], cmap='RdBu')
plt.title('Imag final image')
plt.colorbar()
plt.subplot(2, 2, 3)
plt.imshow(np.abs(img[0] + 1.j * img[1])**2, cmap='inferno')#, norm=LogNorm())
plt.title('Intensity from E-field, final image')
plt.colorbar()
plt.subplot(2, 2, 4)
plt.imshow(img_intensity, cmap='inferno')#, norm=LogNorm())
plt.title('Intensity form file, final image')
plt.colorbar()